# 作業

### 嘗試用 keras 的 DepthwiseConv2D 等 layers 實做 Inverted Residual Block.
   - depthwise's filter shape 爲 (3,3), padding = same
   - 不需要給 alpha, depth multiplier 參數
   - expansion 因子爲 6

##### 載入套件

In [3]:
from tensorflow.keras.layers import Input, DepthwiseConv2D, Conv2D, BatchNormalization, ReLU, Add
from tensorflow.keras.models import Model

2024-09-12 20:02:03.643006: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


##### 定義 Separable Convolution 函數 (請在此實做)

In [5]:

def InvertedRes(input_tensor, expansion):

    input_shape = input_tensor.shape[-1]
    # Step 1: Expansion phase (1x1 Conv2D)
    x = Conv2D(expansion * input_shape, (1, 1), padding='same', use_bias=False)(input_tensor)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Step 2: Depthwise Convolution (3x3 DepthwiseConv2D)
    x = DepthwiseConv2D((3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Step 3: Projection phase (1x1 Conv2D)
    x = Conv2D(input_shape, (1, 1), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    # Step 4: Add residual connection (if input channels match)
    if input_shape == x.shape[-1]:
        x = Add()([input_tensor, x])
    
    return x

##### 建構模型

In [6]:
input_tensor = Input((64, 64, 3))
output_tensor = InvertedRes(input_tensor, 6)
model = Model(inputs=input_tensor, outputs=output_tensor)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 64, 64, 18)           54        ['input_2[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 64, 64, 18)           72        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 re_lu (ReLU)                (None, 64, 64, 18)           0         ['batch_normalization[0][0